# Создание и нормализация базы данных.

## Подготовка

In [1]:
import pandas as pd
from sqlalchemy import create_engine, text

import os
from dotenv import load_dotenv

import warnings
warnings.filterwarnings("ignore")

# Креды для подключения к PostgreSQL. Пользователя и пароль берём из .env
user = os.getenv("POSTGRES_USER")
password = os.getenv("POSTGRES_PASSWORD")
host = "localhost"
port = 5432
database = "db1"

# Создаём движок для работы с базой
engine = create_engine(f"postgresql+psycopg2://{user}:{password}@{host}:{port}/{database}")

# Функция для создания схем и таблиц
def execute_query(sql):
    with engine.connect() as conection:
        try:
            conection.execute(text(sql))
            conection.commit()
            print("Запрос успешно выполнен! 🎉🎉🎉")
        except Exception as e:
            print(f"При выполнении запроса возникла ошибка: {e}")

Читаем первый лист с транзакциями

In [2]:
transactions = pd.read_excel("customer_and_transaction.xlsx", sheet_name="transaction")
transactions.head()

,transaction_id,product_id,customer_id,transaction_date,online_order,order_status,brand,product_line,product_class,product_size,list_price,standart_cost
0,1,2,2950,2017-02-25,False,Approved,Solex,Standard,medium,medium,71.49,53.62
1,2,3,3120,2017-05-21,True,Approved,Trek Bicycles,Standard,medium,large,2091.47,388.92
2,3,37,402,2017-10-16,False,Approved,OHM Cycles,Standard,low,medium,1793.43,248.82
3,4,88,3135,2017-08-31,False,Approved,Norco Bicycles,Standard,medium,medium,1198.46,381.10
4,5,78,787,2017-10-01,True,Approved,Giant Bicycles,Standard,medium,large,1765.30,709.48


Проверяем пропуски

In [3]:
transactions.isna().sum()

transaction_id        0
product_id            0
customer_id           0
transaction_date      0
online_order        360
order_status          0
brand               197
product_line        197
product_class       197
product_size        197
list_price            0
standart_cost       197
dtype: int64

Читаем второй лист с клиентами

In [4]:
customers = pd.read_excel("customer_and_transaction.xlsx", sheet_name="customer")
customers.head()

,customer_id,first_name,last_name,gender,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,address,postcode,state,country,property_valuation
0,1,Laraine,Medendorp,F,1953-10-12 00:00:00,Executive Secretary,Health,Mass Customer,N,Yes,060 Morning Avenue,2016,New South Wales,Australia,10
1,2,Eli,Bockman,Male,1980-12-16 00:00:00,Administrative Officer,Financial Services,Mass Customer,N,Yes,6 Meadow Vale Court,2153,New South Wales,Australia,10
2,3,Arlin,Dearle,Male,1954-01-20 00:00:00,Recruiting Manager,Property,Mass Customer,N,Yes,0 Holy Cross Court,4211,QLD,Australia,9
3,4,Talbot,NaN,Male,1961-10-03 00:00:00,NaN,IT,Mass Customer,N,No,17979 Del Mar Point,2448,New South Wales,Australia,4
4,5,Sheila-kathryn,Calton,Female,1977-05-13 00:00:00,Senior Editor,NaN,Affluent Customer,N,Yes,9 Oakridge Court,3216,VIC,Australia,9


Так же проверяем пропуски

In [5]:
customers.isna().sum()

customer_id                0
first_name                 0
last_name                125
gender                     0
DOB                       87
job_title                506
job_industry_category    656
wealth_segment             0
deceased_indicator         0
owns_car                   0
address                    0
postcode                   0
state                      0
country                    0
property_valuation         0
dtype: int64

## Задание 1. Продумать структуру базы данных и отрисовать в редакторе

Структура БД концептуально будет выглядеть следующим образом  
  
![Если изображение не загрузилось, то схема БД находится в папке проекта](Схема_БД.png)  
  
Однако предварительно нужно преобразовать данные в нужный формат

## Задание 2. Нормализовать базу данных (1НФ — 3НФ), описав, к какой нормальной форме приводится таблица и почему таблица в этой нормальной форме изначально не находилась.

Приведём наши данные к 2НФ, в соответствии со схемой выше. Изначальная таблица находилась в 1НФ потому что все атрибуты были простыми, однако часть столбцов на листе _transaction_ зависела не от первичного ключа `transaction_id`, а от другого столбца - `product_id`. Поэтому для перехода к 2НФ разделим таблицы _transactions_ и _products_

Для начала выделим датафрейм для таблицы products. Чтобы логика работала, нужно чтобы сочетание `brand` + `product_line` + `product_class` + `product_size` + `standart_cost` было уникальным для каждого `product_id`. Проверим

In [6]:
products_columns = ["product_id", "brand", "product_line", "product_class", "product_size", "standart_cost"]
print("Уникальных product_id:", transactions["product_id"].nunique())
print("Уникадьных продуктов по факту:", transactions.groupby(products_columns).ngroups )

Уникальных product_id: 101
Уникадьных продуктов по факту: 203


Видим, что продуктов на самом деле больше, чем id для нашей таблицы. В таком случае сгенерируем свой id

In [7]:
products = transactions.groupby(products_columns, as_index=False)["transaction_id"].count()\
                        .drop(columns=["product_id", "transaction_id"])\
                        .rename_axis(index="id")\
                        .reset_index()
products

,id,brand,product_line,product_class,product_size,standart_cost
0,0,Giant Bicycles,Standard,medium,large,528.43
1,1,Giant Bicycles,Standard,medium,medium,173.18
2,2,Norco Bicycles,Road,medium,medium,376.84
3,3,Norco Bicycles,Road,medium,medium,407.54
4,4,Norco Bicycles,Standard,low,medium,290.41
...,...,...,...,...,...,...
198,198,Trek Bicycles,Standard,high,medium,215.03
199,199,OHM Cycles,Standard,medium,medium,770.89
200,200,Trek Bicycles,Road,low,small,1531.42
201,201,Norco Bicycles,Road,medium,medium,206.35


Приведём таблицу `transactons` к нужному нам виду

In [8]:
# Оставляем только нужные нам поля
transactions = transactions.drop(columns=products_columns)

#Преобразуем поле online_order в бинарное (0/1)
transactions["online_order"] = transactions["online_order"].replace({False:0, True:1})

# Переименовываем колонку transaction_id в id
transactions = transactions.rename(columns={"transaction_id":"id"})

# Смотрим результат
transactions.head(1)

,id,customer_id,transaction_date,online_order,order_status,list_price
0,1,2950,2017-02-25,0.0,Approved,71.49


Переходим к обработке таблицы `customers`

In [9]:
# Конверстируем DOB в дату
customers["DOB"] = pd.to_datetime(customers["DOB"])

# Перводим ответы Да/Нет к бинарному видут
customers["deceased_indicator"] = customers["deceased_indicator"].replace({"N":0, "Y":1})
customers["owns_car"] = customers["owns_car"].replace({"No":0, "Yes":1})

# Переименовываем колонку customer_id в id
customers = customers.rename(columns={"customer_id":"id"})

# Смотрим результат
customers.head(1)

,id,first_name,last_name,gender,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,address,postcode,state,country,property_valuation
0,1,Laraine,Medendorp,F,1953-10-12,Executive Secretary,Health,Mass Customer,0,1,060 Morning Avenue,2016,New South Wales,Australia,10


## Задание 3. Создать все таблицы в DBeaver, указав первичные ключи к таблицам, правильные типы данных, могут ли поля быть пустыми или нет

Создаём схему store

In [10]:
execute_query("CREATE SCHEMA store")

Запрос успешно выполнен! 🎉🎉🎉


Создаём таблицу `products`

In [11]:
sql = """
CREATE TABLE store.products (
    id SERIAL PRIMARY KEY,
    brand VARCHAR(100),
    product_line VARCHAR(50),
    product_class VARCHAR(50),
    product_size VARCHAR(50),
    standard_cost DECIMAL(10, 2)
)
"""

execute_query(sql)

Запрос успешно выполнен! 🎉🎉🎉


Создаём таблицу `customers`

In [12]:
sql = """
CREATE TABLE store.customers (
    id SERIAL PRIMARY KEY,
    first_name VARCHAR(100),
    last_name VARCHAR(100),
    gender VARCHAR(30),
    dob DATE,
    job_title TEXT,
    job_industry_category TEXT,
    wealth_segment VARCHAR(50),
    deceased_indicator SMALLINT,
    owns_car SMALLINT,
    address TEXT,
    postcode VARCHAR(30),
    state TEXT,
    country VARCHAR(100),
    property_valuation SMALLINT
)
"""

execute_query(sql)

Запрос успешно выполнен! 🎉🎉🎉


Создаём таблицу `transactions`

In [13]:
sql = """
CREATE TABLE store.transactions (
    id SERIAL PRIMARY KEY,
    product_id INT,
    customer_id INT,
    transaction_date DATE,
    online_order SMALLINT,
    order_status VARCHAR(50),
    list_price DECIMAL(10, 2)
)
"""

execute_query(sql)

Запрос успешно выполнен! 🎉🎉🎉


## Задание 4. Загрузить данные в таблицы в соответствии с созданной структурой

In [14]:
# Загружаем данные по клиентам
customers.to_sql(schema="store", name="customers", con=engine, if_exists="replace", index=False)

# Проверяем
pd.read_sql("SELECT * FROM store.customers LIMIT 5", con=engine)

,id,first_name,last_name,gender,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,address,postcode,state,country,property_valuation
0,1,Laraine,Medendorp,F,1953-10-12,Executive Secretary,Health,Mass Customer,0,1,060 Morning Avenue,2016,New South Wales,Australia,10
1,2,Eli,Bockman,Male,1980-12-16,Administrative Officer,Financial Services,Mass Customer,0,1,6 Meadow Vale Court,2153,New South Wales,Australia,10
2,3,Arlin,Dearle,Male,1954-01-20,Recruiting Manager,Property,Mass Customer,0,1,0 Holy Cross Court,4211,QLD,Australia,9
3,4,Talbot,None,Male,1961-10-03,None,IT,Mass Customer,0,0,17979 Del Mar Point,2448,New South Wales,Australia,4
4,5,Sheila-kathryn,Calton,Female,1977-05-13,Senior Editor,None,Affluent Customer,0,1,9 Oakridge Court,3216,VIC,Australia,9


In [15]:
# Загружаем данные по продуктам
products.to_sql(schema="store", name="products", con=engine, if_exists="replace", index=False)

# Проверяем
pd.read_sql("SELECT * FROM store.products LIMIT 5", con=engine)

,id,brand,product_line,product_class,product_size,standart_cost
0,0,Giant Bicycles,Standard,medium,large,528.43
1,1,Giant Bicycles,Standard,medium,medium,173.18
2,2,Norco Bicycles,Road,medium,medium,376.84
3,3,Norco Bicycles,Road,medium,medium,407.54
4,4,Norco Bicycles,Standard,low,medium,290.41


In [16]:
# Загружаем данные по транзакциям
transactions.to_sql(schema="store", name="transactions", con=engine, if_exists="replace", index=False)

# Проверяем
pd.read_sql("SELECT * FROM store.transactions LIMIT 5", con=engine)

,id,customer_id,transaction_date,online_order,order_status,list_price
0,1,2950,2017-02-25,0.0,Approved,71.49
1,2,3120,2017-05-21,1.0,Approved,2091.47
2,3,402,2017-10-16,0.0,Approved,1793.43
3,4,3135,2017-08-31,0.0,Approved,1198.46
4,5,787,2017-10-01,1.0,Approved,1765.30
